<a href="https://colab.research.google.com/github/SDS-AAU/DSBA-2021/blob/master/static/notebooks/DSBA21_W1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv('https://stacks.stanford.edu/file/druid:yg821jf8611/yg821jf8611_la_new_orleans_2020_04_01.csv.zip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,raw_row_number,date,time,location,lat,lng,district,zone,subject_age,subject_race,subject_sex,officer_assignment,type,arrest_made,citation_issued,warning_issued,outcome,contraband_found,contraband_drugs,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_stop,vehicle_color,vehicle_make,vehicle_model,vehicle_year,raw_actions_taken,raw_subject_race
0,1,2010-01-01,01:11:00,NaN,NaN,NaN,6,E,26.0,black,female,6th District,vehicular,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,NaN,TRAFFIC VIOLATION,BLACK,DODGE,CARAVAN,2005.0,NaN,BLACK
1,9087,2010-01-01,01:29:00,NaN,NaN,NaN,7,C,37.0,black,male,7th District,vehicular,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,NaN,TRAFFIC VIOLATION,BLUE,NISSAN,MURANO,2005.0,NaN,BLACK
2,9086,2010-01-01,01:29:00,NaN,NaN,NaN,7,C,37.0,black,male,7th District,vehicular,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,NaN,TRAFFIC VIOLATION,BLUE,NISSAN,MURANO,2005.0,NaN,BLACK
3,267,2010-01-01,14:00:00,NaN,NaN,NaN,7,I,96.0,black,male,7th District,vehicular,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,NaN,TRAFFIC VIOLATION,GRAY,JEEP,GRAND CHEROKEE,2003.0,NaN,BLACK
4,2,2010-01-01,02:06:00,NaN,NaN,NaN,5,D,17.0,black,male,5th District,NaN,False,False,False,NaN,NaN,NaN,NaN,False,False,False,False,NaN,CALL FOR SERVICE,NaN,NaN,NaN,NaN,NaN,BLACK


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512092 entries, 0 to 512091
Data columns (total 32 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   raw_row_number      512092 non-null  object 
 1   date                512088 non-null  object 
 2   time                512092 non-null  object 
 3   location            416106 non-null  object 
 4   lat                 260408 non-null  float64
 5   lng                 260408 non-null  float64
 6   district            512092 non-null  object 
 7   zone                512092 non-null  object 
 8   subject_age         499306 non-null  float64
 9   subject_race        500362 non-null  object 
 10  subject_sex         500362 non-null  object 
 11  officer_assignment  511969 non-null  object 
 12  type                362185 non-null  object 
 13  arrest_made         512092 non-null  bool   
 14  citation_issued     512092 non-null  bool   
 15  warning_issued      512092 non-nul

In [8]:
data['datetime'] = data['date'].str.cat(data['time'], ' ')

In [11]:
data.index = pd.to_datetime(data['datetime'])

In [14]:

data.index

DatetimeIndex(['2010-01-01 01:11:00', '2010-01-01 01:29:00',
               '2010-01-01 01:29:00', '2010-01-01 14:00:00',
               '2010-01-01 02:06:00', '2010-01-01 02:06:00',
               '2010-01-01 02:06:00', '2010-01-01 02:06:00',
               '2010-01-01 02:06:00', '2010-01-01 02:06:00',
               ...
               '2017-12-31 00:28:00', '2017-12-31 00:28:00',
               '2017-12-31 00:35:00', '2017-12-31 12:40:00',
               '2017-12-31 00:48:00', '2017-12-31 00:48:00',
               '2017-12-31 00:48:00', '2017-12-31 00:48:00',
               '2017-12-31 00:48:00', '2017-12-31 12:49:00'],
              dtype='datetime64[ns]', name='datetime', length=512092, freq=None)

## Ideas to look into stuff?

- Crosstabs / Distribution of Sex / Race
- Searches conducted
- Drugs / weapons found?
- Expensive cars vs cheap cars - proxy by vehicle age (option)
- Sex / Race / Age vs stop-outcome


In [18]:
# single distrubution / count

data.subject_race.value_counts(normalize=True)

black                     0.699132
white                     0.259218
hispanic                  0.026962
asian/pacific islander    0.007581
unknown                   0.006423
other                     0.000684
Name: subject_race, dtype: float64

In [32]:
data.groupby(['subject_race','reason_for_stop']).size()

subject_race            reason_for_stop       
asian/pacific islander  CALL FOR SERVICE             378
                        CITIZEN CONTACT               64
                        CRIMINAL VIOLATION           125
                        FLAGGED DOWN                  16
                        JUVENILE VIOLATION             1
                        OTHER                        119
                        PRESENT AT CRIME SCENE         1
                        SUSPECT PERSON               134
                        SUSPECT VEHICLE               22
                        TRAFFIC VIOLATION           2933
black                   CALL FOR SERVICE           48879
                        CITIZEN CONTACT            11232
                        CRIMINAL VIOLATION         16069
                        FLAGGED DOWN                2355
                        JUVENILE VIOLATION          2833
                        OTHER                      17871
                        PRESENT AT CRIME 

In [72]:
pd.crosstab(data.subject_race, data.reason_for_stop, normalize='columns')

reason_for_stop,CALL FOR SERVICE,CITIZEN CONTACT,CRIMINAL VIOLATION,FLAGGED DOWN,JUVENILE VIOLATION,OTHER,PRESENT AT CRIME SCENE,SUSPECT PERSON,SUSPECT VEHICLE,TRAFFIC VIOLATION
subject_race,,,,,,,,,,
asian/pacific islander,0.005392,0.004049,0.004166,0.003605,0.000321,0.004916,0.001269,0.002110,0.003965,0.010370
black,0.697255,0.710526,0.535562,0.530644,0.910347,0.738288,0.794416,0.761864,0.760317,0.697728
hispanic,0.025748,0.018029,0.022697,0.034475,0.005463,0.020821,0.030457,0.014344,0.021445,0.031791
other,0.000542,0.000443,0.000500,0.000451,0.000000,0.000578,0.001269,0.000315,0.000000,0.000866
unknown,0.004037,0.003036,0.002366,0.004957,0.000321,0.003801,0.003807,0.001464,0.003784,0.009122
white,0.267025,0.263917,0.434709,0.425868,0.083548,0.231595,0.168782,0.219902,0.210488,0.250123


In [37]:
pd.crosstab(data.subject_race, data.arrest_made, normalize='index')

arrest_made,False,True
subject_race,,
asian/pacific islander,0.925125,0.074875
black,0.801034,0.198966
hispanic,0.844785,0.155215
other,0.894737,0.105263
unknown,0.920971,0.079029
white,0.835324,0.164676


In [85]:
data.groupby(['subject_race', 'subject_sex'])['arrest_made'].mean()

subject_race            subject_sex
asian/pacific islander  female         0.049738
                        male           0.085757
black                   female         0.169189
                        male           0.211642
hispanic                female         0.117513
                        male           0.162509
other                   female         0.090909
                        male           0.110236
unknown                 female         0.075356
                        male           0.079692
white                   female         0.134734
                        male           0.176940
Name: arrest_made, dtype: float64

In [41]:
data.groupby('subjec_sex')['arrest_made'].mean()

subject_sex
female    0.158225
male      0.199035
Name: arrest_made, dtype: float64

In [78]:
data[data.search_conducted == True].groupby('subject_race')['contraband_weapons'].sum()/3113

subject_race
asian/pacific islander    0.002891
black                     0.880822
hispanic                  0.012849
other                     0.000000
unknown                   0.002249
white                     0.099904
Name: contraband_weapons, dtype: float64

In [77]:
data[data.search_conducted == True].contraband_weapons.sum()

3113

In [69]:
data.groupby('district')['vehicle_year'].mean().sort_values()

district
5        2003.450792
5        2003.520488
4        2003.806462
3        2003.956794
1        2003.957323
4        2003.966812
7        2003.970504
7        2003.998879
3        2004.020548
8        2004.056494
1        2004.142627
8        2004.341650
2        2004.589421
2        2004.607631
6        2005.188022
6        2005.502055
1|7              NaN
3|2              NaN
5|3|3            NaN
6|2              NaN
Name: vehicle_year, dtype: float64

In [86]:
data[data.district.isin([5,6])].groupby('district').search_conducted.mean()

district
5    0.124911
6    0.170545
Name: search_conducted, dtype: float64

In [88]:
data[data.district.isin([5,6])].groupby('district').subject_race.value_counts(normalize=True)

district  subject_race          
5         black                     0.758880
          white                     0.214093
          hispanic                  0.018758
          unknown                   0.004710
          asian/pacific islander    0.003085
          other                     0.000473
6         black                     0.734431
          white                     0.225178
          hispanic                  0.024931
          unknown                   0.009662
          asian/pacific islander    0.005188
          other                     0.000609
Name: subject_race, dtype: float64

### Some Geoplotting

In [97]:
import folium
m = folium.Map(tiles='Stamen Toner', location=[55.6832933, 12.5169813], zoom_start=16)

In [99]:
m

In [101]:
# subset data (we need lon/lat)
geoplot_data = data.sample(20).dropna(subset=['lat'])

In [104]:
print(geoplot_data.lat.mean())
print(geoplot_data.lng.mean())

29.968287416666666
-90.07055595833333


In [105]:
m = folium.Map(location=[30, -90], zoom_start=12)

tooltip = "I'm a police stop"

# iterate over all rows
for row in geoplot_data.iterrows():
  folium.Marker([row[1].lat, row[1].lng], 
                popup=f"<b>{row[1].subject_age}</b> <b>{row[1].subject_race}</b> <b>{row[1].reason_for_stop}</b> <b>{row[1].outcome}</b>", 
                tooltip=tooltip).add_to(m)


m